In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
# !wget https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/albert-base-2020-04-10.tar.gz
# !tar -zxf albert-base-2020-04-10.tar.gz
!ls albert-base-2020-04-10

config.json			       model.ckpt-400000.meta
model.ckpt-400000.data-00000-of-00001  sp10m.cased.v10.model
model.ckpt-400000.index		       sp10m.cased.v10.vocab


In [5]:
from albert import modeling
from albert import optimization
from albert import tokenization
import tensorflow as tf
import numpy as np

In [6]:
tokenizer = tokenization.FullTokenizer(
      vocab_file='albert-base-2020-04-10/sp10m.cased.v10.vocab', do_lower_case=False,
      spm_model_file='albert-base-2020-04-10/sp10m.cased.v10.model')


INFO:tensorflow:loading sentence piece model


In [7]:
albert_config = modeling.AlbertConfig.from_json_file('albert-base-2020-04-10/config.json')
albert_config

In [8]:
import json

with open('dataset.json') as fopen:
    data = json.load(fopen)

In [9]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [10]:
import re
from unidecode import unidecode

def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [11]:
from tqdm import tqdm

for i in tqdm(range(len(train_X))):
    train_X[i] = cleaning(train_X[i])
    
for i in tqdm(range(len(test_X))):
    test_X[i] = cleaning(test_X[i])

100%|██████████| 5029/5029 [00:01<00:00, 2863.78it/s]


In [12]:
MAX_SEQ_LENGTH = 510

def get_tokenize(strings):
    input_ids, input_masks = [], []
    for text in tqdm(strings):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a) > MAX_SEQ_LENGTH - 2:
            tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)
        input_ids.append(input_id)
        input_masks.append(input_mask)
    return input_ids, input_masks

In [13]:
train_input_ids, train_input_masks = get_tokenize(train_X)
test_input_ids, test_input_masks = get_tokenize(test_X)

100%|██████████| 5029/5029 [00:20<00:00, 241.23it/s]


In [14]:
epoch = 10
batch_size = 16
warmup_proportion = 0.1
num_train_steps = int(len(train_X) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [15]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.MASK = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.AlbertModel(
            config=albert_config,
            is_training=training,
            input_ids=self.X,
            input_mask=self.MASK,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        output_layer = tf.layers.dense(
            output_layer,
            albert_config.hidden_size,
            activation=tf.tanh,
            kernel_initializer=create_initializer())
        self.logits_seq = tf.layers.dense(output_layer, dimension_output,
                                         kernel_initializer=create_initializer())
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [16]:
BERT_INIT_CHKPNT = 'albert-base-2020-04-10/model.ckpt-400000'

In [17]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)





Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:++++++ warmup starts at step 0, for 3235 steps ++++++
INFO:tensorflow:using adamw

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from albert-base-2020-04-10/model.ckpt-400000


In [18]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [19]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = train_Y[i: index]
        batch_mask = train_input_masks[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.MASK: batch_mask
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = test_Y[i: index]
        batch_mask = test_input_masks[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.MASK: batch_mask
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/3236 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.853571
time taken: 2090.0366172790527
epoch: 0, training loss: 0.423202, training acc: 0.802746, valid loss: 0.364440, valid acc: 0.853571



train minibatch loop:   0%|          | 0/3236 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.853571, current acc: 0.881746
time taken: 2081.8950345516205
epoch: 1, training loss: 0.300792, training acc: 0.875637, valid loss: 0.360250, valid acc: 0.881746



train minibatch loop:   0%|          | 0/3236 [00:00<?, ?it/s]

epoch: 2, pass acc: 0.881746, current acc: 0.886706
time taken: 2082.3744621276855
epoch: 2, training loss: 0.214198, training acc: 0.922512, valid loss: 0.437990, valid acc: 0.886706



test minibatch loop: 100%|██████████| 315/315 [01:11<00:00,  4.43it/s, accuracy=1, cost=7.53e-5]   

time taken: 2082.8626000881195
epoch: 3, training loss: 0.155375, training acc: 0.954380, valid loss: 0.682126, valid acc: 0.882937

break epoch:4



In [27]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'albert-base-relevancy/model.ckpt')

'albert-base-relevancy/model.ckpt'

In [21]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate,
    training = False
)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'albert-base-relevancy/model.ckpt')

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:++++++ warmup starts at step 0, for 3235 steps ++++++
INFO:tensorflow:using adamw
INFO:tensorflow:Restoring parameters from albert-base-relevancy/model.ckpt


In [22]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/LayerNorm/gamma',
 'bert/encoder/embedding_hidden_mapping_in/kernel',
 'bert/encoder/embedding_hidden_mapping_in/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/bias',
 'bert/encoder/transformer/group_0/inner_group_0/Lay

In [23]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = test_Y[i: index]
    batch_mask = test_input_masks[i: index]
    batch_mask = pad_sequences(batch_mask, padding='post')
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            model.MASK: batch_mask
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 315/315 [01:10<00:00,  4.44it/s]


In [24]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['not relevant', 'relevant'],
        digits = 5
    )
)

              precision    recall  f1-score   support

not relevant    0.93970   0.75176   0.83529      1990
    relevant    0.85627   0.96841   0.90889      3039

    accuracy                        0.88268      5029
   macro avg    0.89798   0.86008   0.87209      5029
weighted avg    0.88928   0.88268   0.87977      5029



In [25]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [26]:
freeze_graph('albert-base-relevancy', strings)

INFO:tensorflow:Restoring parameters from albert-base-relevancy/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 29 variables.
INFO:tensorflow:Converted 29 variables to const ops.
3242 ops in the final graph.
